In [1]:
!pip install transformers datasets sentencepiece

In [43]:
import pandas as pd
import transformers

In [44]:
data = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
data['score'] = data['score'].astype(float)
data = data[['full_text', 'score']]

In [45]:
data['full_text'].str.split().str.len().max()

1656

# Model

In [46]:
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification
from datasets import Dataset

tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')
model = BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base', num_labels=1)

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['full_text'], padding="max_length", truncation=True, max_length=1728)

# Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.2)
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.remove_columns(['full_text'])
dataset = dataset.rename_column('score', 'labels')
dataset.set_format('torch')

Map:   0%|          | 0/13845 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [50]:
from transformers import Trainer, TrainingArguments
import torch
import gc

training_args = TrainingArguments(
    report_to="none",
    output_dir='./results',
    
    eval_strategy='steps',
    eval_steps=50,
    logging_steps=50,
    
    save_strategy='epoch',
    
    learning_rate=2e-5,
    per_device_train_batch_size=3,  # Adjust batch size to fit memory constraints
    per_device_eval_batch_size=3,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=8,  # Accumulate gradients
    local_rank=-1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

gc.collect()
torch.cuda.empty_cache()


In [ ]:
# Train the model on TPU
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


# Test